<a href="https://colab.research.google.com/github/EllaScheltinga/Deep-Learning-Group-75/blob/main/AEGNN_(ncaltech101).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setting device and importing dependencies**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#David
import sys
sys.path.append('/content/drive/MyDrive/Onderwijs/TU Delft: MSc RO/AEGNN/src')

In [ ]:
# Ella
import sys
sys.path.append('/content/drive/MyDrive/AEGNN/src')

In [ ]:
!pip install -q torch_geometric
!pip install pytorch-lightning
!pip install -q torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torch.utils.tensorboard import SummaryWriter
#Additional Setup to use Tensorboard
!pip install -q tensorflow
%load_ext tensorboard

### **Loading and preprocessing data**

In [ ]:
!unzip -u "/content/drive/MyDrive/Onderwijs/TU Delft: MSc RO/AEGNN/data/Caltech101.zip" -d /content/

Archive:  /content/drive/MyDrive/Onderwijs/TU Delft: MSc RO/AEGNN/data/Caltech101.zip


In [ ]:
!unzip -q /content/drive/MyDrive/Caltech101.zip -d /content/ 

replace /content/Caltech101/accordion/image_0001.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
def read_label(raw_file):
    return raw_file.split("/")[-2]

In [ ]:
import glob
import os

glob_ = glob.glob(os.path.join("/content/Caltech101/*","*"), recursive=True)
classes = {}
item_id = 0
for item in glob_:
  class_ = read_label(item)
  if class_ not in classes.keys():
    classes[class_] = item_id
    item_id += 1

# classes = {
#     'umbrella': 0,
#     'butterfly': 1,
#     'barrel': 2
# }

In [ ]:
from utils.load import load_object
from utils.preprocess import pre_transform_all

data = []
NUM_CLASSES = 10

#for key in ['umbrella', 'butterfly', 'barrel']:
for key in list(classes.keys())[:NUM_CLASSES]:
  print(key)
  raw = load_object(key)
  data += pre_transform_all(raw, classes[key], device)

llama
kangaroo
ketch
soccer_ball
laptop
wrench
wheelchair
bass
starfish
menorah


In [ ]:
from torch.utils.data import random_split as Split
from torch_geometric.loader import DataLoader

BATCH_SIZE  = 4
validation, test, train = Split(data, [0.1, 0.1, 0.8])
validation, test, train = DataLoader(validation.dataset, batch_size = BATCH_SIZE),\
                          DataLoader(test.dataset, batch_size = BATCH_SIZE),\
                          DataLoader(train.dataset, batch_size = BATCH_SIZE)

### **Training model**

In [ ]:
import torch.optim
from models.recognition import RecognitionModel

criterion = torch.nn.CrossEntropyLoss().cuda()
rm = RecognitionModel(network="graph_res", dataset="ncaltech101", num_classes=NUM_CLASSES, img_shape=(240,180)).to(device)
optimizer = torch.optim.Adam(rm.parameters(), lr = 0.1)

In [ ]:
def training(model, data):
  seen = 0
  correct = 0
  # Create a writer to write to Tensorboard
  for item in iter(data):
      item = item.to(device)
      optimizer.zero_grad()
      out = model.forward(item)
      loss = criterion(out, item.y)
      loss.backward()
      pred = out.max(dim=1)[1]
      seen += len(item)
      correct += pred.eq(item.y).sum().item()
      optimizer.step()

  return correct / seen

In [ ]:
def validating(model, data):
    seen = 0
    correct = 0
    for item in iter(data):
        item = item.to(device)
        with torch.no_grad():
            out = model.forward(item)
            pred = out.max(dim=1)[1]
            seen += len(item)
            correct += pred.eq(item.y).sum().item()

    return correct / seen

In [ ]:
def testing(model, data):
    seen = 0
    correct = 0
    for item in iter(data):
        item = item.to(device)
        with torch.no_grad():
            out = model.forward(item)
            pred = out.max(dim=1)[1]
            seen += len(item)
            correct += pred.eq(item.y).sum().item()

    return correct / seen

In [ ]:
from tqdm import tqdm

highest_acc = 0
num_epoch = 15
model = rm

av_hist = np.zeros((num_epoch, 3))
writer = SummaryWriter()


for epoch in tqdm(range(num_epoch)):
  train_acc = training(model = model, data = train)
  validation_acc = validating(model = model, data = validation)

  av_hist[epoch]=np.array([epoch, train_acc, validation_acc])

  # Write metrics to Tensorboard
  writer.add_scalar('Train Accuracy', train_acc , epoch)
  writer.add_scalar('Validation Accuracy', validation_acc , epoch)


  if validation_acc > highest_acc:
      torch.save(model.state_dict(), 'best_model.pt')
      highest_acc = validation_acc

  print(f'Epoch {epoch:03d}, train acc.: {train_acc:.4f}, val. acc.: {validation_acc:.4f}',
                              flush = True)
print('\nFinished.')
writer.flush()
writer.close()

  0%|          | 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch_geometric/utils/scatter.py:93: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(f"The usage of `scatter(reduce='{reduce}')` "


Epoch 000, train acc.: 0.7433, val. acc.: 0.1096


  7%|▋         | 1/15 [00:09<02:18,  9.90s/it]

Epoch 001, train acc.: 0.7313, val. acc.: 0.1096


 13%|█▎        | 2/15 [00:17<01:47,  8.27s/it]

...

 47%|████▋     | 7/15 [00:41<00:41,  5.22s/it]

Epoch 007, train acc.: 0.7527, val. acc.: 0.1965


 53%|█████▎    | 8/15 [00:45<00:35,  5.03s/it]

Epoch 008, train acc.: 0.7901, val. acc.: 0.2500


 60%|██████    | 9/15 [00:51<00:30,  5.08s/it]

### **Running model**

In [ ]:
# Run Tensorboard
# !rm -r runs
%tensorboard --logdir runs/

<IPython.core.display.Javascript object>